In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from skimage.feature import hog
import pickle
from tqdm import tqdm
from imp import reload

In [6]:
import hn_detect
reload(hn_detect)
import hn_detect

In [3]:
# Load training set label
labels = pd.read_csv('./train_label.csv')

In [4]:
labels.head()

,name,height,left,top,width,label
0,1.png,219,246,77,81,1
1,1.png,219,323,81,96,9
2,2.png,32,77,29,23,2
3,2.png,32,98,25,26,3
4,3.png,15,17,5,8,2


In [5]:
# Get name of all images in training set
train_path = '../data/train/'
train_images = os.listdir(train_path)

In [10]:
def get_cropped(labels, i, option =0):
    """
    Get number bouding box in a image.
    """
    image = hn_detect.pre_process(train_path, labels['name'][i])
    
    # Check if left position is < 0
    x1 = labels['top'][i]
    if (x1 < 0): 
        return None
    
    x2 = x1 + labels['height'][i]
    
    # Check if top position is < 0
    y1 = labels['left'][i]
    if (y1 < 0): 
        return None
    
    y2 = y1 + labels['width'][i]
    
    # Get number bounding box
    cropimg = image[x1:x2,y1:y2]
    
    # Check if bounding box is too small
    if (option == 0): #16x32
        if (((x2-x1) < 32 ) or ((y2-y1)<16)):
            #print('return NONE')
            return None
    elif(option == 1): #16x8
        if (((x2-x1) < 16 ) or ((y2-y1)<8)):
            #print('return NONE')
            return None
    return cropimg

In [8]:
def resize_img(img, option = 0):
    """
    Resize image to a fixed size
    """
    #option 0: 16x32
    #option 1: 8x16
    if option == 0:
        dim = (16,32)
    elif option == 1:
        dim = (8,16)
    #print(img.shape)
    img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return img_resized


In [17]:
filename = 'HOG_16x32'

# Number of HOG feature
num_desc = 756

# Initialize
tosave = np.zeros((len(labels), num_desc))
count = 0

# For all number in training set
for i in tqdm(range(len(labels))):
    # Get bounding box
    I = get_cropped(labels, i)
    if (I is None):
        count = count+1
        continue
    
    # Resize bounding box
    I = resize_img(I, option =0)
    
    # Get bounding box HOG feature
    desc= hog(I, orientations=9, pixels_per_cell = (4,4), cells_per_block = (2,2))
    tosave[i,:] = desc


100%|██████████| 145560/145560 [05:14<00:00, 462.69it/s]


In [19]:
print(tosave.shape)
print(count)

(145560, 756)
70288


In [20]:
# Initialize
tosavenew = np.array([])

# Remove too small bounding boxes
for i in tqdm(range(len(labels))):
    if np.all(tosave[i,:] == np.zeros(num_desc)):
        continue
    tosavenew = np.append(tosavenew, tosave[i,:])
print(tosavenew.shape)
tosavenew = tosavenew.reshape(-1,num_desc)

100%|██████████| 145560/145560 [3:23:01<00:00, 11.95it/s](56905632,)



In [22]:
tosavenew.shape

(75272, 756)

In [23]:
# Save data
fileObj = open('./16x32/' + filename, 'wb')
pickle.dump(tosavenew, fileObj)
fileObj.close()